In [1]:
import pandas as pd
import os
import numpy as np
import geopy
import geopandas
import matplotlib.pyplot as plt
import math
import re

# Residential Tree Planting project data prep

## Loading and formatting data

In [2]:
#Rename excel files
old_names = list(os.listdir('./data'))

for i in old_names:
    if '.xlsx' not in i or '$' in i:
        old_names.remove(i)

old_names

['boundaries',
 'fall_2023.xlsx',
 'fall_2024.xlsx',
 'fall_2025.xlsx',
 'spring_2023.xlsx',
 'spring_2024_1.xlsx',
 'spring_2024_2.xlsx',
 'spring_2025.xlsx']

In [3]:
new_names = []
for i in old_names:
    new_name = i.lower().replace(" ","_").strip()
    new_name = new_name.replace("casey_", "").replace("tree_","").replace("trees_","").replace("planting_","")
    new_names.append(new_name)

new_names

['boundaries',
 'fall_2023.xlsx',
 'fall_2024.xlsx',
 'fall_2025.xlsx',
 'spring_2023.xlsx',
 'spring_2024_1.xlsx',
 'spring_2024_2.xlsx',
 'spring_2025.xlsx']

In [4]:
# Read files into df
dfs = {}
for i in new_names:
    if 'xlsx' in i:
        dfs[i] = pd.read_excel(("./data/"+i), header=0)

C:\Users\pmedina\AppData\Local\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [5]:
dfs.keys()

dict_keys(['fall_2023.xlsx', 'fall_2024.xlsx', 'fall_2025.xlsx', 'spring_2023.xlsx', 'spring_2024_1.xlsx', 'spring_2024_2.xlsx', 'spring_2025.xlsx'])

## Formatting data functions

### Column names cleaning

In [6]:
# Variable name formatting
def clean_column_names(df):
    df.columns = (df.columns
               .str.strip()
               .str.lower()
               .str.replace(" ", "_")
               .str.replace(";","")
               .str.replace("\\t","")
               .str.replace("[()€$]", "",

                            regex=True))
    return df

### Update species fields 

In [7]:
pd.set_option('mode.chained_assignment', None)
def update_species(df):
    species_columns = ['species_1', 'species_2', 'species_3', 'species_4']
    df_columns = list(df.columns)
    test_value = 0
    for name in species_columns:
        if name in df_columns:
            test_value += 1
    if test_value >= 3:
        return df
    else:
        for index, row in df.iterrows():
            if ',' in row['species_1']:
                species = df.loc[index, 'species_1'].split(',')
                species_1 = species[0]
                species_2 = species[1]
                df.loc[index, 'species_1'] = species_1
                df.loc[index, 'species_2'] = species_2
            elif '2' in row['species_1']:
                species = row['species_1'].strip().replace('2','').replace('()','')
                df.loc[index, 'species_1'] = species
                df.loc[index, 'species_2'] = species
            else:
                row['species_2'] = ''
        return df

In [8]:
def transform_species(df):
    # Create variables for the function
    new_df = df[["city_district", "address", "species_1", "year"]].copy() # New dataframe
    new_df.rename(columns={"species_1":"species"}, inplace=True) # Rename column species_1 to species
    initial_value = False # Set initial value
    extra_species = ["species_2", "species_3", "species_4"] # Extra species columns
    counter = 0
    
    # If the dataframe has any of the extra species columns set the initial value to True 
    for column in df.columns: 
        if "species" in column:
            counter += 1
    
    #df.drop("species_1", axis=1, inplace=True) # Drop species 1 since it's already included in the new df
    
    # If the initial value is True, then
    if counter > 1:
        for index, row in df.drop("species_1", axis=1,).iterrows(): # For each indexed row in the input dataframe
            for column in df.columns: # For each column in the input dataframe
                if column in extra_species: # If the column is in the extra species columns list
                    if (str(row[column]).lower() != "nan") and (len(str(row[column])) > 2): # If the row is not empty
                        # Create a new row with city district, address, species, and year as fields
                        new_row = pd.DataFrame({"city_district": [row["city_district"]], 
                                                "address": [row["address"]], 
                                                "species": [row[column]], 
                                                "year": [row["year"]]}) 
                        # Add the new row to the copied new dataframe
                        new_df = pd.concat([new_df, new_row], ignore_index=True)
        # Return the new dataframe
        return new_df
    else:
        new_df = df.rename(columns={"species_1": "species"})
        return new_df

### Drop variables containing PII and unnecesary ones

In [9]:
def column_clean(df):
    del_words = ['application', 'name','email','casey','form','contact','area','inspection','date','phone','quantity','insp']
    column_list = list(df.columns)
    columns_to_drop = []
    new_df = df

    if 'unnamed:_1' in column_list:
        new_column_names = list(new_df.loc[0,])
        new_df.columns = new_column_names
        new_df = clean_column_names(new_df)
        new_df = new_df.iloc[1:]
        column_list = list(new_df.columns)
        
    for column_name in column_list:
        for word in del_words:
            if word in column_name:
                if column_name not in columns_to_drop:
                    columns_to_drop.append(column_name)

    new_df.drop(columns=columns_to_drop, axis = 1, inplace = True)
     
    species_columns = 1
    
    for column_name in column_list:
        if "species" in column_name:
            # species_columns.append(column_name)
            new_name = 'species_' + str(species_columns)
            new_df.rename(columns={column_name:new_name}, inplace = True)
            species_columns += 1
        elif "district_#" in column_name or 'city_dist' in column_name:
            new_df.rename(columns={column_name:"city_district"}, inplace = True)
        elif "address" in column_name:
            new_df.rename(columns={column_name:"address"}, inplace = True)
        elif "district" in column_name:
            new_df.rename(columns={column_name:"city_district"}, inplace = True)
        
            
    return new_df
            

### Create main function

In [10]:
def format_table(df, year):
    if 'unnamed:_1' in list(df.columns):
            new_df = column_clean(df)
            new_df = update_species(new_df)
            new_df['year'] = year
            new_df = transform_species(new_df)
            return new_df
    else:
            new_df = clean_column_names(df)
            new_df = column_clean(new_df)
            new_df = update_species(new_df)
            new_df['year'] = year
            new_df = transform_species(new_df)
            return new_df

### Geolocate function

In [11]:
from geopy.geocoders import Nominatim
 
geolocator = Nominatim(user_agent="myapp")

def geocode(table, field):
    lat=[] #Latitude list
    lon=[] #Longitude list
    progress = 0 #Progress percentage
    total = len(table)
    #For each row
    for index, row in table.iterrows():
        #Geolocate passing the address field for the geolocator
        location = geolocator.geocode(row[field], timeout=None)
        if location is not None: #If the geolocator returns a valid value
            latitude = location.latitude #extract the latitude value
            longitude = location.longitude #Extract the longitude value
            lat.append(latitude) #Append it to the lat list
            lon.append(longitude) #Append it to the long list
        else: #If the function returns an invalid value
        #Set the values a 0,0
            lat.append(0)
            lon.append(0)
        progress += 1
        percentage = progress/total*100
        #print("%.2f %%" % percentage, end="\r", flush=True)
        print(" " * 50, end="\r", flush=True) # Clear the line
        print(f"{percentage:.2f} %", end="\r", flush=True)
    #Return the list
    return lat, lon

## Format tables

In [12]:
spring_2023 = dfs['spring_2023.xlsx']
fall_2023 = dfs['fall_2023.xlsx']
spring_2024_2 = dfs['spring_2024_2.xlsx']
fall_2025 = dfs['fall_2025.xlsx']
fall_2024 = dfs['fall_2024.xlsx']
spring_2025 = dfs['spring_2025.xlsx']
spring_2024_1 = dfs['spring_2024_1.xlsx']

### Spring 2023

In [13]:
spring_2023.head(2)

,Application Date; Planting on Resid Property,Area,District #,First and Last Name:,Email Address:,Phone Number:,Address:,Preferred Contact,Consultation date,Casey Trees date planted,City inspection date,Species planted,Notes (Casey Trees)
0,2022-05-14 21:50:25,Daniels Pk E,1,Kirstin Showalter,kirstinsd@gmail.com,7346492987,5121 Kenesaw St,Email,2023-02-28,NaN,NaN,"A.Hornbeam, Wil Oak",match
1,2022-11-11 13:10:39,Daniels Pk W,1,Anne Tolbert,ktolbertmd@gmail.com,301-613-3496,4908 Hollywood Rd,Email,2023-03-01,NaN,NaN,(2) Willow Oaks,no map


In [14]:
spring_2023 = format_table(spring_2023, 2023)

In [15]:
spring_2023

,city_district,address,species,year
0,1,5121 Kenesaw St,A.Hornbeam,2023
1,1,4908 Hollywood Rd,Willow Oaks,2023
2,1,4822 Erie Street,S. Magnolia,2023
3,1,5017 Laguna Road,S. Magnolia,2023
4,1,9118 Autoville Dr.,Am. Witch Hazel,2023
5,2,8302 49TH AVE,Baldcypress,2023
6,3,4614 Amherst Rd,Catalpa,2023
7,3,7207 Rhode Island Ave,Am. Hornbeam,2023
8,3,5911 Bryn Mawr Rd,S. Magnolia,2023
9,4,9116 Saint Andrews Pl,Am. Holly,2023


### Fall 2023

In [16]:
fall_2023.head(2)

,Casey Tree Planting on Private Property FALL 2023,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,Application Date; Planting on Residential Prop...,Area,District #,First and Last Name:,Email Address:,Phone Number,Address:,Preferred Contact,Consultation date,Casey Trees date planted,City inspection date,Species planted,Hold Harmless Form completed
1,2022-06-25 00:00:00,CP Estates,3,Andrew Stephens,andrewstephens11@gmail.com,202-550-5392,7512 Wellesley Dr,email,2023-06-08 00:00:00,2024-10-30 00:00:00,2023-11-17 00:00:00,American Linden,X


In [17]:
fall_2023 = format_table(fall_2023, 2023)
#fall_2023 = fall_2023.drop(0)
fall_2023.head(5)

,city_district,address,species,year
1,3,7512 Wellesley Dr,American Linden,2023
2,4,3711 Marlbrough Way,Red Maple,2023
3,2,4803 Ruatan St,Tulip Poplar,2023
4,1,4806 Lackawanna St,Southern Magnolia,2023
5,1,9815 52nd Pl,Tulip Poplar,2023


### Spring 2024 - 1

In [18]:
spring_2024_1.head(2)

,Application Date; Planting on Residential Property,City District,First and Last Name:,Email Address:,Phone Number,Address:,Consultation date,Casey pl,City insp,Species planted,Hold Harmless Form completed
0,2023-09-17,1,Anning Cheng,anningc@yahoo.com,757-268-8522,5109 Iroquois St,2023-12-08,2024-04-05,2024-04-23,Red Maple,X
1,2023-10-08,1,Kim Tolbert,ktolbertmd@gmail.com,301-613-3496,4908 Hollywood Rd,2023-12-08,2024-04-05,2024-04-23,"Beech, Red Maple",X


In [19]:
spring_2024_1 = format_table(spring_2024_1, 2024)
spring_2024_1.head(5)

,city_district,address,species,year
0,1,5109 Iroquois St,Red Maple,2024
1,1,4908 Hollywood Rd,Beech,2024
2,1,4810 Lackawanna St,Dogwood,2024
3,1,5106 Lackawanna St,Southern Magnolia,2024
4,1,9735 53rd Ave,Red Maple,2024


### Spring 2024 - 2

In [20]:
spring_2024_2.head(2)

,Casey Tree Planting on Private Property SPRING 2024 (MAY),Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Application Date; Planting on Residential Prop...,City District,First and Last Name:,Email Address:,Phone Number,Address:,Consultation date,Casey Trees date planted,City inspection date,Species planted,Hold Harmless Form completed
1,2024-02-28 00:00:00,1,Jeff Miller,jeffmillerworld@gmail.com,443-764-6083,9609 50th Place,2024-02-28 00:00:00,2024-04-10 00:00:00,2024-04-23 00:00:00,"Beech, Swamp White Oak",X


In [21]:
spring_2024_2 = format_table(spring_2024_2, 2024)
spring_2024_2.head(10)

,city_district,address,species,year
0,1,9609 50th Place,Beech,2024
1,1,9540 Rhode Island Ave,Persimmon,2024
2,1,5121 Kennebunk Terr,Freeman Maple,2024
3,2,4811 Ruatan St,Swamp White Oak,2024
4,3,7510 Wellesley Dr,Redbud,2024
5,3,4611 Drexel Rd,Persimmon,2024
6,4,3739 Marlbrough Way,Am. Holly,2024
7,4,7321 Radcliffe Dr,Am Holly,2024
8,NaN,NaN,Total = 14 trees,2024
9,1,9609 50th Place,Swamp White Oak,2024


### Fall 2024

In [22]:
fall_2024.head(2)

,City Dist,First and Last Name:,Email Address:,Phone Number:,Address of the Tree,Species 1,Species 2,Species 3,Species 4,Notes,Casey Trees date planted,City Inspetion date
0,1,William Crane,frunpro@outlook.com,2022465897,8611 Rhode Island Ave,American holly,Redbud,Swamp white oak,NaN,NaN,2024-11-19 00:00:00,2024-11-19 00:00:00
1,1,James Milmoe,milmoej2007@hotmail.com,4434667799,8709 48th Ave,American holly,Nuttall oak,NaN,NaN,NaN,2024-11-19 00:00:00,2024-11-19 00:00:00


In [23]:
fall_2024 = format_table(fall_2024, 2024)
fall_2024.head(5)

,city_district,address,species,year
0,1,8611 Rhode Island Ave,American holly,2024
1,1,8709 48th Ave,American holly,2024
2,1,9629 52nd Ave,Redbud,2024
3,1,6917 Carleton Ter,Apple serviceberry,2024
4,1,9621 Narragansett Pkwy,Persimmon,2024


### Spring 2025

In [24]:
spring_2025.head(2)

,City Dist,First and Last Name:,Email Address:,Phone Number:,Address of the Tree,Species 1,Species 2,Species 3,Species 4,Notes,Casey Trees date planted,City Inspetion date
0,1,Silvia Alvarado,marylicastillo@gmail.com,202-460-5615,9406 52nd Ave,Arborvitae,Arborvitae,NaN,NaN,NaN,NaN,NaN
1,1,Brian Mayers,soccerplaya37@gmail.com,443-534-4109,5202 Lackawanna St,Southern Magnolia,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
spring_2025 = format_table(spring_2025, 2025)
spring_2025.head(5)

,city_district,address,species,year
0,1,9406 52nd Ave,Arborvitae,2025
1,1,5202 Lackawanna St,Southern Magnolia,2025
2,1,9037 49th Pl,NS Holly,2025
3,1,4712 Nantucket Rd,Am Elm,2025
4,1,5022 Geronimo St,Redbud,2025


### Fall 2025

In [26]:
fall_2025.head(2)

,First and Last Name:,Email Address:,Phone Number:,District,Address,Species 1,Species 2,Tree quantity
0,Adam Claudy,atclaudy@gmail.com,301-356-0282,Disctrict 1,9400 49th Ave,Redbud,NaN,1
1,Aida Xhepa,aidaxhepa@gmail.com,202-445-4361,Disctrict 1,9605 52nd Ave,Red Maple,NaN,1


In [27]:
fall_2025 = format_table(fall_2025, 2025)
fall_2025.head()

,city_district,address,species,year
0,Disctrict 1,9400 49th Ave,Redbud,2025
1,Disctrict 1,9605 52nd Ave,Red Maple,2025
2,Disctrict 3,5012 Roanoke Pl,Apple Serviceberry,2025
3,Disctrict 2,8619 Rhode Island Ave,Ironwood (Hophornbeam),2025
4,Disctrict 1,9505 52nd Ave,Sweetbay Magnolia,2025


In [28]:
fall_2025

,city_district,address,species,year
0,Disctrict 1,9400 49th Ave,Redbud,2025
1,Disctrict 1,9605 52nd Ave,Red Maple,2025
2,Disctrict 3,5012 Roanoke Pl,Apple Serviceberry,2025
3,Disctrict 2,8619 Rhode Island Ave,Ironwood (Hophornbeam),2025
4,Disctrict 1,9505 52nd Ave,Sweetbay Magnolia,2025
...,...,...,...,...
56,Disctrict 3,7307 Baylor Ave,Overcup Oak,2025
57,Disctrict 2,8700 49th Ave,Holly,2025
58,Disctrict 3,7400 Dartmouth Ave,American Hornbeam,2025
59,Disctrict 1,9727 53rd Ave,Brackens,2025


### Consolidate data

In [62]:
consolidated_data = pd.concat([spring_2023,fall_2023,spring_2024_1,spring_2024_2,fall_2024,spring_2025,fall_2025])

In [63]:
consolidated_data['address'] = consolidated_data['address'] + ", College Park MD"

In [64]:
consolidated_data = consolidated_data.reset_index() #Reset index
del consolidated_data['index'] #Drop old index

In [66]:
consolidated_data.head()

,city_district,address,species,year
0,1,"5121 Kenesaw St, College Park MD",A.Hornbeam,2023
1,1,"4908 Hollywood Rd, College Park MD",Willow Oaks,2023
2,1,"4822 Erie Street, College Park MD",S. Magnolia,2023
3,1,"5017 Laguna Road, College Park MD",S. Magnolia,2023
4,1,"9118 Autoville Dr. , College Park MD",Am. Witch Hazel,2023


### Consolidate species names

In [67]:
species_names = list(consolidated_data['species'].unique())

In [68]:
consolidated_data.index[consolidated_data['species']=='2 Southern Magnolia']

Index([80], dtype='int64')

Correct names with numbers

In [69]:
consolidated_data.loc[80,]

city_district                                         2
address          8323 Rhode Island Ave, College Park MD
species                             2 Southern Magnolia
year                                               2024
Name: 80, dtype: object

In [71]:
consolidated_data.at[80,'species'] = 'Southern Magnolia'

In [72]:
consolidated_data.index[consolidated_data['species']=='TBD']

Index([184, 225, 226, 227], dtype='int64')

In [73]:
consolidated_data.at[184,'species'] = 'Eastern Red Cedar'
consolidated_data.at[225,'species'] = 'Eastern Red Cedar'
consolidated_data.at[226,'species'] = 'Eastern Red Cedar'
consolidated_data.at[227,'species'] = 'Eastern Red Cedar'

### Reaload the unique species values

In [74]:
species_names = []

In [75]:
species_fields = []
species_fields.append(list(consolidated_data['species'].unique()))

In [76]:
for list_item in species_fields:
    for item in list_item:
        species_names.append(item)

### Create dictionary for the new species name

In [77]:
species_names_keys = {}

for name in species_names:
    new_name = ''
    if 'A.' in str(name):
        new_name = name.replace('A.','American ').lstrip(' ')
        species_names_keys[name] = new_name.title()
    elif 'Am.' in str(name):
        new_name = name.replace('Am.','American').lstrip(' ')
        species_names_keys[name] = new_name.title()
    elif 'Amer ' in str(name):
        new_name = name.replace('Amer ','American ').lstrip(' ')
        species_names_keys[name] = new_name.title()
    elif 'Americanholly' in str(name):
        new_name = 'American Holly'
        species_names_keys[name] = new_name.title()
    elif 'Am ' in str(name):
        new_name = name.replace('Am ','American ').lstrip(' ')
        species_names_keys[name] = new_name.title()
    elif 'S.' in str(name):
        new_name = name.replace('S.','Southern').lstrip(' ')
        species_names_keys[name] = new_name.title()
    elif ' Wil ' in str(name):
        new_name = name.replace(' Wil ','Willow ').lstrip(' ')
        species_names_keys[name] = new_name.title()
    elif 'Oaks' in str(name):
        new_name = name.replace('Oaks','Oak').lstrip(' ')
        species_names_keys[name] = new_name.title().lstrip(' ')
    elif ' Magnol' in str(name):
        new_name = re.sub('Magnol$', 'Magnolia', name)
        species_names_keys[name] = new_name.title()
    elif 'Baldcypress' in str(name):
        new_name = re.sub('Baldcypress$', 'Bald Cypress', name)
        species_names_keys[name] = new_name.title()
    elif 'Honeylocust' in str(name):
        new_name = re.sub('Honeylocust$', 'Honey locust', name)
        species_names_keys[name] = new_name.title()
    elif ' Eastern Redcedar' in str(name):
        new_name = 'Eastern Red cedar'
        species_names_keys[name] = new_name.title()
    elif ' Freemen Maple' in str(name):
        new_name = 'Freeman Maple'
        species_names_keys[name] = new_name.title().lstrip(' ')
    else:
        new_name = str(name).strip()
        species_names_keys[name] = new_name.title().lstrip(' ')
        

### Update values

In [79]:
for index, row in consolidated_data.iterrows():
    if row["species"] in list(species_names_keys.keys()):
        consolidated_data.at[index, "species"] = str(species_names_keys[row["species"]])

In [80]:
consolidated_data

,city_district,address,species,year
0,1,"5121 Kenesaw St, College Park MD",American Hornbeam,2023
1,1,"4908 Hollywood Rd, College Park MD",Willow Oak,2023
2,1,"4822 Erie Street, College Park MD",Southern Magnolia,2023
3,1,"5017 Laguna Road, College Park MD",Southern Magnolia,2023
4,1,"9118 Autoville Dr. , College Park MD",American Witch Hazel,2023
...,...,...,...,...
338,Disctrict 3,"7307 Baylor Ave, College Park MD",Overcup Oak,2025
339,Disctrict 2,"8700 49th Ave, College Park MD",Holly,2025
340,Disctrict 3,"7400 Dartmouth Ave, College Park MD",American Hornbeam,2025
341,Disctrict 1,"9727 53rd Ave, College Park MD",Brackens,2025


In [82]:
file = r"C:\Users\pmedina\Documents\Projects\Colle_Park_Residential_Tree_Planting_PRogram\method-calculating-carbon-sequestration-trees-urban-and-suburban-settings.pdf"

In [99]:
from PyPDF2 import PdfReader

In [100]:
reader = PdfReader(file)

In [101]:
print(len(reader.pages))

16


In [106]:
page = reader.pages[7]
text = page.extract_text()
text

'Method for Calculating Carbon Sequestration by Trees in Urban and Suburban Settings 7Table 1. Common Urban Tree Species\nSpecies Type Growth Species Type Growth \nRate Rate\nAilanthus, Ailanthus altissima Maple, bigleaf, Acer macrophyllum\nAlder, European, Alnus glutinosa\nAsh, green, Fraxinus pennsylvanica\nAsh, mountain, American ,Sorbus americana\nAsh, white, Fraxinus americanaHF H S\nHF H MHF H MHM H MHF H SMaple, Norway, Acer platanoides\nMaple, red, Acer rubrum\nMaple, silver, Acer saccharinum\nMaple, sugar, Acer saccharum\nAspen, bigtooth, Populus grandidentata Mulberry, red, Morus rubra\nAspen, quaking, Populus tremuloides Oak, black, Quercus velutina\nBaldcypress, Taxodium distichum Oak, blue, Quercus douglasii\nBasswood, American, Tilia americana ,O a k ,  b u r ,  Quercus macrocarpa\nBeech, American, Fagus grandifolia Oak, California black, Quercus kelloggiiHM H F\nHF H M\nCF H M\nHF H SHS H S\nBirch, paper (white), Betula papyrifera Oak, California White, Quercus lobata\nB

In [60]:
consolidated_data.to_csv("consolidated.csv")

### Clean coordinates

#### Check the missing lat/lon

In [ ]:
consolidated_data[consolidated_data['lat'] == 0]

##### De Pauw Place

In [ ]:
depauw_pl_3504 = geopandas.tools.geocode('3504 De Pauw Place, College Park MD')
depauw_pl_3504

In [ ]:
depauw_pl_3504_lat = depauw_pl_3504.centroid.y
depauw_pl_3504_lon = depauw_pl_3504.centroid.x
print("Latitude: %.4f \nLongitude: %.4f" % (depauw_pl_3504_lat, depauw_pl_3504_lon))

In [ ]:
consolidated_data.at[57, 'lat'] = depauw_pl_3504_lat
consolidated_data.at[57, 'lon'] = depauw_pl_3504_lon

##### Saint Andrews Place

In [ ]:
saint_andrews_pl = geopandas.tools.geocode('9224 Saint Andrews Pl, College Park MD')
saint_andrews_pl

In [ ]:
saint_andrews_pl_lat = saint_andrews_pl.centroid.y
saint_andrews_pl_lon = saint_andrews_pl.centroid.x
print("Latitude: %.4f \nLongitude: %.4f" % (saint_andrews_pl_lat, saint_andrews_pl_lon))

In [ ]:
consolidated_data.at[154, 'lat'] = saint_andrews_pl_lat
consolidated_data.at[154, 'lon'] = saint_andrews_pl_lon

# Calculate Variables
Will calcualte the following:
* Carbon sequestration
* Rainfall interception
* Energy stored

## Carbon Sequestration

In [34]:
consolidated_data

,city_district,address,species,year
0,1,"5121 Kenesaw St, College Park MD",A.Hornbeam,2023
1,1,"4908 Hollywood Rd, College Park MD",Willow Oaks,2023
2,1,"4822 Erie Street, College Park MD",S. Magnolia,2023
3,1,"5017 Laguna Road, College Park MD",S. Magnolia,2023
4,1,"9118 Autoville Dr. , College Park MD",Am. Witch Hazel,2023
...,...,...,...,...
56,Disctrict 3,"7307 Baylor Ave, College Park MD",Overcup Oak,2025
57,Disctrict 2,"8700 49th Ave, College Park MD",Holly,2025
58,Disctrict 3,"7400 Dartmouth Ave, College Park MD",American Hornbeam,2025
59,Disctrict 1,"9727 53rd Ave, College Park MD",Brackens,2025


## Geocode data

In [33]:
coordinates = geocode(consolidated_data, 'address')

consolidated_data["lat"] = coordinates[0]
consolidated_data["lon"] = coordinates[1]

KeyboardInterrupt: 

### Check outliers

In [ ]:
#Load consolidated dataset as geopandas object
gdf = geopandas.GeoDataFrame(
    consolidated_data, geometry=geopandas.points_from_xy(consolidated_data.lon, consolidated_data.lat), crs="EPSG:4326"
)

In [ ]:
cp = geopandas.read_file("data/boundaries/College_Park_City_Limits.shp")
cp = cp.to_crs(4326)
cp.crs

In [ ]:
ax = cp.plot(color="white", edgecolor="black")

# We can now plot our ``GeoDataFrame``.
gdf.plot(ax=ax, color="red")

plt.show()

Since some data seem to be out of the range, let's look at the coordinates sorted in descending order.

In [ ]:
consolidated_data.sort_values(by='lon', ascending=False).head(5)

In [ ]:
consolidated_data = consolidated_data.drop(index=108)
consolidated_data.sort_values(by='lon', ascending=False).head(5)

In [ ]:
gdf = geopandas.GeoDataFrame(
    consolidated_data, geometry=geopandas.points_from_xy(consolidated_data.lon, consolidated_data.lat), crs="EPSG:4326"
)

In [ ]:
ax = cp.plot(color="white", edgecolor="black")

# We can now plot our ``GeoDataFrame``.
gdf.plot(ax=ax, color="red")

plt.show()

# Save consolidated data

In [ ]:
consolidated_data.to_csv('consolidated_data.csv')

In [ ]:
species_list = []
species_list.append(list(consolidated_data['species_1']))
species_list.append(list(consolidated_data['species_2']))
species_list.append(list(consolidated_data['species_3']))
species_list.append(list(consolidated_data['species_4']))

In [ ]:
species_list[3][2] == 'Nan'

In [ ]:
species = []
for i in species_list:
    for elem in i:
        if elem != 'Nan':
            species.append(elem.lstrip(' '))

In [ ]:
species_df = pd.DataFrame(species, columns=['Species'])

In [ ]:
species_df = species_df.groupby('Species')['Species'].count()

In [ ]:
species_df = pd.DataFrame(species_df)

In [ ]:
species_df.columns = ['Count']
species_df

In [ ]:
species_df.to_csv('total_species.csv')